In [ ]:
#| default_exp filtering

# MatchFilter
The MatchFilter abstract class defines the interface for implementations that rejecting potential matchings of response units with target items based on specified conditions. Example conditions include a minimum similarity score between the response unit and the target item or detection of a specific entailment relation between the response unit and the target item. This class is used to supplement concrete Matcher classes by providing conditions under which matchings can be rejected.

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import List, Union, Dict
import numpy as np

class MatchFilter(ABC):

    """
    Abstract base class for match filtering functionality. 
    To create a custom Filter, inherit from this class and 
    override the filter method.
    """

    @abstractmethod
    def __call__(self, 
               response_units: Union[List[str], List[Dict[str, object]]], 
               target_items: Union[List[str], List[Dict[str, object]]],
               score_matrix: np.ndarray) -> np.ndarray:
        """
        Updates the score matrix based the filter criteria so filtered pairs have a score of -inf.

        Parameters:
            response_units (Union[List[str], List[Dict[str, object]]]): List of response units. 
                Each can be a string (when only text is available) or
                a dictionary (when both text and spans are available).
            target_items (Union[List[str], List[Dict[str, object]]]): List of target items.
                Each can be a string (when only text is available) or
                a dictionary (when both text and spans are available).
            score_matrix (np.ndarray): A matrix of scores between each response unit and target item.

        Returns:
            np.ndarray: The updated score matrix.
        """
        pass

In [ ]:
#| export
from typing import Optional
from response_sequencer.scoring import Scorer

class ThresholdMatchFilter(MatchFilter):
    """
    A filter that filters out pairs with scores below a threshold. 
    
    Either uses provided scores to filter or uses a scorer to generate scores. 
    """
    def __init__(self, threshold: float, scorer: Optional[Scorer] = None):
        self.threshold = threshold
        self.scorer = scorer

    def __call__(self, 
               response_units: Union[List[str], List[Dict[str, object]]], 
               target_items: Union[List[str], List[Dict[str, object]]],
               score_matrix: np.ndarray) -> np.ndarray:
        
        threshold_matrix = self.scorer(response_units, target_items) if self.scorer else score_matrix
        score_matrix[threshold_matrix < self.threshold] = -np.inf

        return score_matrix

In [ ]:
#|export
from typing import Callable

class ConditionMatchFilter(MatchFilter):
    """
    A filter that filters out pairs that do not meet a condition function that takes in a response unit and target item.
    """

    def __init_ (self, condition: Callable[[str, str], bool]):
        self.condition = condition

    def __call__(self,
                 response_units: Union[List[str], List[Dict[str, object]]], 
                 target_items: Union[List[str], List[Dict[str, object]]],
                 score_matrix: np.ndarray) -> np.ndarray:
           
        if isinstance(response_units[0], dict):
             response_units = [ru['text'] for ru in response_units]
        if isinstance(target_items[0], dict):
             target_items = [ti['text'] for ti in target_items]
             
        for i, ru in enumerate(response_units):
            for j, ti in enumerate(target_items):
                if not self.condition(ru, ti):
                    score_matrix[i, j] = -np.inf
                    
        return score_matrix

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()